In [1]:
#%%
import torch
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
import os
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from torch import nn
import torch.optim as optim

# 读取数据
train_dir='/home/tsinghuaee112/Classification/Data/Train'
val_dir='/home/tsinghuaee112/Classification/Data/Val'
transform  = transforms.Compose([
         transforms.Resize((64,64)),
         #transforms.RandomHorizontalFlip(),
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.4, 0.4, 0.4]),
])#读取数据时对数据进行初步处理：大小归一化，值强度归一化
trainset = ImageFolder(train_dir, transform=transform)
valset=ImageFolder(val_dir,transform=transform)

# 深度学习中图片数据一般保存成CxHxW，即通道数x图片高x图片宽
#print(trainset[0][0].size())
#print(valset[0][0].size())
# 0.2和0.4是标准差和均值的近似
#a=transforms.ToPILImage()(trainset[0][0]*0.2+0.4)
#plt.imshow(a)
#plt.show()
#b=transforms.ToPILImage()(valset[0][0]*0.2+0.4)
#plt.imshow(b)
#plt.show()


In [2]:

#%%
#数据划分batch
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=4)
valloader = torch.utils.data.DataLoader(valset, batch_size=32,
                                         shuffle=True, num_workers=4)
classes = ('i2', 'i4', 'i5', 'io','ip','p5','p11','p23','p26','pl5','pl30','pl40','pl50','pl60'
           'pl60','pl80','pn','pne','po','w57')


In [3]:

#%%


#网络模型

#class Net(nn.Module):
#    def __init__(self):
#        super(Net, self).__init__()
#        self.conv1 = nn.Conv2d(3, 6, 5)
#        self.pool = nn.MaxPool2d(2, 2)
#        self.conv2 = nn.Conv2d(6, 16, 5)
#        self.fc1 = nn.Linear(16 * 5 * 5, 120)
#        self.fc2 = nn.Linear(120, 84)
#        self.fc3 = nn.Linear(84, 19) #输出层

#    def forward(self, x):
#        x = self.pool(F.relu(self.conv1(x)))
#        x = self.pool(F.relu(self.conv2(x)))
#        x = x.view(-1, 16 * 5 * 5)
#        x = F.relu(self.fc1(x))
#        x = F.relu(self.fc2(x))
#        x = self.fc3(x)
#        return x

#net = Net()



net=models.resnet18(pretrained=True)
#net = models.resnext50_32x4d(pretrained=True)
num_ftrs=net.fc.in_features
net.fc=nn.Linear(num_ftrs,19)



#定义优化模型
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
scheduler=optim.lr_scheduler.StepLR(optimizer,step_size=10000,gamma=0.2)


#ignored_params= list(map(id,net.fc.parameters()))
#base_params=filter(lambda p: id(p) not in ignored_params, net.parameters())
#optimizer=torch.optim.SGD([
#    {'params':base_params},
#    {'params':net.fc.parameters(),'lr':0.001}
#],lr=0,momentum=0.9)


#%%


os.environ["CUDA_VISIBLE_DEVICES"]="2"
net=net.cuda()
#训练网络
for epoch in range(32):  # loop over the dataset multiple times
    running_loss = 0.0
#    for i, data in enumerate(trainloader, 0):

    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        inputs=inputs.cuda()
        labels=labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()
        scheduler.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')


[1,   100] loss: 1.207
[1,   200] loss: 0.359
[1,   300] loss: 0.251
[2,   100] loss: 0.120
[2,   200] loss: 0.088
[2,   300] loss: 0.106
[3,   100] loss: 0.054
[3,   200] loss: 0.058
[3,   300] loss: 0.058
[4,   100] loss: 0.032
[4,   200] loss: 0.034
[4,   300] loss: 0.034
[5,   100] loss: 0.025
[5,   200] loss: 0.023
[5,   300] loss: 0.025
[6,   100] loss: 0.016
[6,   200] loss: 0.014
[6,   300] loss: 0.014
[7,   100] loss: 0.014
[7,   200] loss: 0.012
[7,   300] loss: 0.010
[8,   100] loss: 0.008
[8,   200] loss: 0.006
[8,   300] loss: 0.008
[9,   100] loss: 0.007
[9,   200] loss: 0.006
[9,   300] loss: 0.005
[10,   100] loss: 0.005
[10,   200] loss: 0.006
[10,   300] loss: 0.005
[11,   100] loss: 0.005
[11,   200] loss: 0.004
[11,   300] loss: 0.005
[12,   100] loss: 0.003
[12,   200] loss: 0.003
[12,   300] loss: 0.003
[13,   100] loss: 0.006
[13,   200] loss: 0.003
[13,   300] loss: 0.004
[14,   100] loss: 0.003
[14,   200] loss: 0.004
[14,   300] loss: 0.003
[15,   100] loss: 0

In [4]:

#%%
PATH = './Resnet_net.pth'
torch.save(net.state_dict(), PATH)
#dataiter = iter(valloader)
#images, labels = dataiter.next()
#第1个batch的分类结果
#print (images)
#print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))
#net = Net()
net=models.resnet18(pretrained=True)
#net = models.resnext50_32x4d(pretrained=True)
num_ftrs=net.fc.in_features
net.fc=nn.Linear(num_ftrs,19)
net.load_state_dict(torch.load(PATH))
#outputs = net(images)
#_, predicted = torch.max(outputs, 1)
#print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
#                              for j in range(32)))


<All keys matched successfully>

In [5]:

#统计正确率
correct = 0
total = 0
with torch.no_grad():
    for data in valloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

#%%
#统计各类别的正确率
class_correct = list(0. for i in range(19))
class_total = list(0. for i in range(19))
with torch.no_grad():
    for data in valloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        for i in range(labels.shape[0]):
            label = labels[i]
            class_correct[label] += (predicted[i] == labels[i]).sum().item()
            class_total[label] += 1

for i in range(19):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of the network on the test images: 98 %
Accuracy of    i2 : 98 %
Accuracy of    i4 : 100 %
Accuracy of    i5 : 99 %
Accuracy of    io : 96 %
Accuracy of    ip : 100 %
Accuracy of    p5 : 98 %
Accuracy of   p11 : 95 %
Accuracy of   p23 : 94 %
Accuracy of   p26 : 97 %
Accuracy of   pl5 : 98 %
Accuracy of  pl30 : 99 %
Accuracy of  pl40 : 98 %
Accuracy of  pl50 : 98 %
Accuracy of pl60pl60 : 99 %
Accuracy of  pl80 : 98 %
Accuracy of    pn : 99 %
Accuracy of   pne : 100 %
Accuracy of    po : 92 %
Accuracy of   w57 : 97 %
